# INSTAGRAM ANALYTICS - ABENDI

### OBJETIVO

Coletar de forma automática dados através de APIs da plataforma para a geração de dashboards no Power BI para facilitar a tomada de decisão do Marketing e Diretoria.


### INFORMAÇÕES GERAIS

1. Acessar o site https://developers.facebook.com/
2. Id do Aplicativo: 547482393462303
3. Chave secreta do Aplicativo: e0651da1344fff14fefcb9d981ebeb4b
4. Permissões do Token de Acesso: 
    - public_profile
    - pages_show_list
    - instagram_basic
    - instagram_manage_comments
    - instagram_manage_insights
    - pages_manage_ads
    - pages_manage_metadata
    - pages_read_engagement
    - pages_read_user_content
    
### Métricas para cada tipo de produto
- feed = ''
- ad = ''
- story = 'exits,impressions,reach,replies,taps_forward,taps_back'
- reels = 'comments,likes,plays,reach,saved,shares,total_interactions'

### Métrica para cada tipo de publicação
- carousel = 'carousel_album_engagement,carousel_album_impressions,carousel_album_reach,carousel_album_saved,carousel_album_video_views'
- image = 'engagement,impressions,reach,saved'
- video = 'video_views,engagement,impressions,reach,saved'

**OBS.: O INSTAGRAM INSIGHTS FOI DISPONIBILIZADO PARA AS PUBLICAÇÕES A PARTIR DE 2016. PORTANTO, PUBLICAÇÕES ANTERIORES NÃO POSSUEM INSIGHTS CONTABILIZADOS**

**OBS.: A VARIÁVEL SHORT_TOKEN É COLETADA APENAS COM A FERRAMENTA "EXPLORADORA DA GRAPH API"**

## 1 - IMPORTAÇÕES DE PACOTES

In [1]:
import requests
import json
import datetime
from datetime import date, timedelta
import pandas as pd
import pyodbc
import smtplib
import email.message

## 2 - FUNÇÕES

In [2]:
def getAllPosts():
    fields = 'id,permalink,caption,media_type,timestamp'
    
    link = f'{end_point_base}{ig_id}/media?fields={fields}&access_token={access_token}'
    response = requests.get(link).json()

    link_post = []
    descricao = []
    tipo_post = []
    data_post = []
    id_post = []

    pagingUrl = []

    #Primeira Página
    for post in response['data']:
        id_post.append(post['id'])
        link_post.append(post['permalink'])
        descricao.append(post['caption'])
        tipo_post.append(post['media_type'])
        data_post.append(post['timestamp'])
    
    pagingUrl.append(response['paging']['next'])
    
    # Demais Páginas
    while True:
        try:
            response = requests.get(pagingUrl[-1])
            data = response.json()
            for post in data['data']:
                id_post.append(post['id'])
                link_post.append(post['permalink'])
                try:
                    descricao.append(post['caption'])
                except:
                    descricao.append('Sem descrição')
                tipo_post.append(post['media_type'])
                data_post.append(post['timestamp'])
            pagingUrl.append(data['paging']['next'])
        except:
            break
    
    # Criando o Dataframe
    df = pd.DataFrame({'Data':day,'Id':id_post,'Link do Post':link_post,'Tipo':tipo_post,'Data da Postagem':data_post})
    
    print('Posts coletados com sucesso !')
        
    return df

def getPostAddInfo():
    fields = 'id,like_count,media_product_type'
    id_post = []
    like_count_post = []
    tipo_produto_post = [] 

    for id in df1['Id']:
        link = f'https://graph.facebook.com/v15.0/{id}?fields={fields}&access_token={access_token}'
        response = requests.get(link).json() 
        id_post.append(response['id'])
        like_count_post.append(response['like_count'])
        tipo_produto_post.append(response['media_product_type'])
    
    df = pd.DataFrame({'Id':id_post,'Likes':like_count_post,'Produto':tipo_produto_post})
    
    print('Informações adicionais dos Posts coletados com sucesso !')
    
    return df

def getDataExpireAccessToken():
    # Coletando informações do Token de Acesso de tempo curto

    # Requests API:
    # https://graph.facebook.com/debug_token?input_token={input-token}&access_token={valid_access-token}

    link = f'{graph_domain}debug_token?input_token={access_token}&access_token={access_token}'
    response = requests.get(link).json()
    
    expire_at = datetime.datetime.fromtimestamp(response['data']['data_access_expires_at'])
    
    return expire_at

def getLongAccessToken():
    
    # Gerando Token de Acesso de Longa Duração (Normalmente a duração desse tipo de token é de 3 meses)
    
    # Requests API:
    # https://graph.facebook.com/{graph-api-version}/oauth/access_token?grant_type=fb_exchange_token&client_id={app-id}&client_secret={app-secret}&fb_exchange_token={your-access-token}

    link = f'{end_point_base}oauth/access_token?grant_type=fb_exchange_token&client_id={app_id}&client_secret={app_secret}&fb_exchange_token={short_token}'
    response = requests.get(link).json()
    
    return response['access_token']

## 3 - VARIÁVEIS

In [3]:
short_token = 'DIGITE SEU TOKEN DE ACESSO TEMPORÁRIO.'
access_token = 'DIGITE SEU TOKEN.'

app_id = 'DIGITE SEU ID DO APP.'
app_secret = 'DIGITE SEU CÓDIGO DE ACESSO AO APP.'

graph_domain = 'https://graph.facebook.com/'
graph_version = 'v15.0/'
end_point_base = graph_domain + graph_version

ig_id = 'DIGITE SEU ID DA PÁGINA DO INSTAGRAM'
ig_username = 'DIGITE O USER NAME DA PÁGINA DO INSTAGRAM'

day = date.today() - timedelta(days=1)
day = str(day.strftime('%d/%m/%Y')) 

data = date.today()
data = str(data.strftime('%d/%m/%Y')) 

remetente = 'DIGITE O REMETENTE.' 
senha = 'DIGITE A SENHA DE PERMISSÃO DE USO DO APP DEFINIDA NAS CONFIGURAÇÕES DE SEGURANÇA DA SUA CONTA GOOGLE.'

## 4 - RENOVAÇÃO DO TOKEN

**Requests API:**

https://graph.facebook.com/{graph-api-version}/oauth/access_token?grant_type=fb_exchange_token&client_id={app-id}&client_secret={app-secret}&fb_exchange_token={your-access-token}

In [4]:
# Verificando a data de expiração do token de acesso
expire_at = getDataExpireAccessToken()

# Definindo a data de renovação do token de acesso
token_renew_at = expire_at - timedelta(days=3)

# Verificando se a data de expiração é a mesma data de renovação do token de acesso. Caso seja a mesma data, o token de acesso é renovado.
if expire_at == token_renew_at:
    link = f'{end_point_base}oauth/access_token?grant_type=fb_exchange_token&client_id={app_id}&client_secret={app_secret}&fb_exchange_token={access_token}'
    response = requests.get(link, ).json()
    access_token = response['access_token']
    print('Token Renovado !')
else:
    print('Não é necessário renovação. Token ainda está no prazo de validade.')

Não é necessário renovação. Token ainda está no prazo de validade.


## 5 - STORIES

**Requests API:**

https://graph.facebook.com/{api-version}/{ig-media-id}/stories?access_token={access-token}

**OS STORIES FICAM DISPONÍVEIS POR APENAS 24 HORAS APENAS**

In [5]:
# Coletando os stories do dia anterior
link = f'https://graph.facebook.com/v15.0/17841400454077387/stories?access_token={access_token}'
response = requests.get(link).json()

In [6]:
# Armazenando os stories em uma lista
id_stories = []

for i in range(len(response['data'])):
    id_stories.append(response['data'][i]['id'])
    
if len(id_stories) == 0:
    id_stories.append(0)

In [7]:
# Criando coluna Stories no Dataframe para Stories

df_stories = pd.DataFrame({'Data':day,'Id Stories':id_stories})

In [8]:
# Salvando os dados

df_stories.to_csv(r'C:\Users\Ricardo\Desktop\tb_insta_stories.csv', encoding='utf-8',index=False)

In [ ]:
print('Stories coletados com sucesso !')

## 6 - PUBLICAÇÕES

In [9]:
df1 = getAllPosts()

Posts coletados com sucesso !


### 6.1 - PUBLICAÇÕES : INFORMAÇÕES ADICIONAIS

In [10]:
df2 = getPostAddInfo()

# Merge
df3 = pd.merge(df1, df2, on='Id', left_index=False)

Informações adicionais dos Posts coletados com sucesso !


### 6.2- PUBLICAÇÕES: INSIGHTS

**Requests API:**

https://graph.facebook.com/{api-version}/{ig-media-id}/insights?metric={metric}&access_token={access-token}



**Insights disponíveis pela API:**

métricas: 'engagement,impressions,reach,saved'

In [11]:
# Engajamento
id_engagement = []
engagement = []

for id in df3['Id']:
    link = f'{graph_domain}{id}/insights?metric=engagement&access_token={access_token}'
    response = requests.get(link).json()
    id_engagement.append(id)
    try:
        engagement.append(response['data'][0]['values'][0].get('value'))
    except:
        engagement.append(0)
            
df_engagement = pd.DataFrame({'Id':id_engagement,'Engajamento':engagement})

df4 = pd.merge(df3, df_engagement, on='Id', left_index=False)

print('Engajamento dos Posts coletados com sucesso !')

Engajamento dos Posts coletados com sucesso !


In [12]:
# Impressões
id_impressions = []
impressions = []

for id in df4['Id']:
    link = f'{graph_domain}{id}/insights?metric=impressions&access_token={access_token}'
    response = requests.get(link).json()
    id_impressions.append(id)
    try:
        impressions.append(response['data'][0]['values'][0].get('value'))
    except:
        impressions.append(0)
            
df_impressions = pd.DataFrame({'Id':id_impressions,'Impressões':impressions})

df5 = pd.merge(df4,df_impressions, on='Id',left_index=False)

print('Impressões dos Posts coletados com sucesso !')

Impressões dos Posts coletados com sucesso !


In [13]:
# Alcance
id_reach = []
reach = []

for id in df5['Id']:
    link = f'{graph_domain}{id}/insights?metric=reach&access_token={access_token}'
    response = requests.get(link).json()
    id_reach.append(id)
    try:
        reach.append(response['data'][0]['values'][0].get('value'))
    except:
        reach.append(0)
            
df_reach = pd.DataFrame({'Id':id_reach,'Alcance':reach})

df6 = pd.merge(df5,df_reach, on='Id', left_index=False)

print('Alcance dos Posts coletados com sucesso !')

Alcance dos Posts coletados com sucesso !


In [14]:
# Salvos
id_saved = []
saved = []

for id in df6['Id']:
    link = f'{graph_domain}{id}/insights?metric=saved&access_token={access_token}'
    response = requests.get(link).json()
    id_saved.append(id)
    try:
        saved.append(response['data'][0]['values'][0].get('value'))
    except:
        saved.append(0)
            
df_saved = pd.DataFrame({'Id':id_saved,'Salvos':saved})

df7 = pd.merge(df6,df_saved, on='Id', left_index=False)

print('Salvos dos Posts coletados com sucesso !')

Salvos dos Posts coletados com sucesso !


### 6.3 - PUBLICAÇÕES: INSIGHTS - REELS

**Requests API:**

https://graph.facebook.com/{api-version}/{ig-media-id}/insights?metric={metric}&access_token={access-token}
    
    
**Insights disponíveis para REELS:**

métricas:'comments,likes,plays,reach,saved,shares,total_interactions'

In [15]:
# Comentários, Exibição do Vídeo, Compartilhamentos e Total de Interações

id_reels = []
comments = []
plays = []
shares = []
total_interactions = []

metrics = 'comments,plays,shares,total_interactions'

for i in df7.index:
    id = df7['Id'][i]
    link = f'{graph_domain}{graph_version}{id}/insights?metric={metrics}&access_token={access_token}'
    response = requests.get(link).json()
    id_reels.append(id)
    if df7['Produto'][i] == 'REELS':
        comments.append(response['data'][0]['values'][0].get('value'))
        plays.append(response['data'][1]['values'][0].get('value'))
        shares.append(response['data'][2]['values'][0].get('value'))
        total_interactions.append(response['data'][3]['values'][0].get('value'))
    else:
        comments.append(0)
        plays.append(0)
        shares.append(0)
        total_interactions.append(0)
            
df_reels = pd.DataFrame({'Id':id_reels,'Comentários':comments,'Exibição dos Vídeos':plays,'Compartilhamentos':shares,
                         'Total de Interações':total_interactions})

df8 = pd.merge(df7,df_reels, on='Id', left_index=False)

print('Comentários, Exibição do Vídeo, Compartilhamentos e Total de Interações dos Reels coletados com sucesso !')

Comentários, Exibição do Vídeo, Compartilhamentos e Total de Interações dos Reels coletados com sucesso !


### 6.4 - PUBLICAÇÕES: INSIGHTS - CAROUSEL

**Requests API:**

https://graph.facebook.com/{api-version}/{ig-media-id}/insights?metric={metric}&access_token={access-token}

**Insights disponíveis na API:**

métricas: 'carousel_album_engagement,carousel_album_impressions,carousel_album_reach,carousel_album_saved,carousel_album_video_views'

In [16]:
# Engajamento do Carrossel, Impressões do Carrossel, Alcance do Carrossel, Salvos do Carrossel, Visualizações de Vídeo do Carrossel
id_carousel = []
carousel_album_engagement = []
carousel_album_impressions = []
carousel_album_reach = []
carousel_album_saved = []
carousel_album_video_views = []

metrics = 'carousel_album_engagement,carousel_album_impressions,carousel_album_reach,carousel_album_saved,carousel_album_video_views'

for i in df8.index:
    id = df8['Id'][i]
    link = f'{graph_domain}{graph_version}{id}/insights?metric={metrics}&access_token={access_token}'
    response = requests.get(link).json()
    id_carousel.append(id)
    if df8['Tipo'][i] == 'CAROUSEL_ALBUM':
        carousel_album_engagement.append(response['data'][0]['values'][0].get('value'))
        carousel_album_impressions.append(response['data'][1]['values'][0].get('value'))
        carousel_album_reach.append(response['data'][2]['values'][0].get('value'))
        carousel_album_saved.append(response['data'][3]['values'][0].get('value'))
        carousel_album_video_views.append(response['data'][4]['values'][0].get('value'))
    else:
        carousel_album_engagement.append(0)
        carousel_album_impressions.append(0)
        carousel_album_reach.append(0)
        carousel_album_saved.append(0)
        carousel_album_video_views.append(0)
            
df_carousel = pd.DataFrame({'Id':id_carousel,'Engajamento do Carrossel':carousel_album_engagement,
                            'Impressões do Carrossel':carousel_album_impressions,
                            'Alcance do Carrossel':carousel_album_reach,'Salvos do Carrossel':carousel_album_saved,
                            'Visualizações de Vídeo do Carrossel':carousel_album_video_views})
df9 = pd.merge(df8,df_carousel, on='Id', left_index=False)

print('Engajamento, Impressões, Alcance, Salvos, Visualizações de Vídeo do Carrossel coletados com sucesso !')

Engajamento, Impressões, Alcance, Salvos, Visualizações de Vídeo do Carrossel coletados com sucesso !


### 6.5 - PUBLICAÇÕES: INSIGHTS - IMAGE

**Requests API:**

https://graph.facebook.com/{api-version}/{ig-media-id}/insights?metric={metric}&access_token={access-token}

**Insights disponíveis para IMAGE:**

métricas:'engagement,impressions,reach,saved'

### 6.6 - PUBLICAÇÕES: INSIGHTS - VÍDEOS

**Requests API:**

https://graph.facebook.com/{api-version}/{ig-media-id}/insights?metric={metric}&access_token={access-token}

**Insights disponíveis para VIDEO:**

métricas:'video_views,engagement,impressions,reach,saved'


**OBS.: AS METRICAS PARA VÍDEOS NÃO SÃO COMPATÍVEIS COM OS REELS**

In [17]:
# Visualizações dos Vídeos
id_videos = []
video_views = []

metric = 'video_views'

for i in df9.index:
    id = df9['Id'][i]
    link = f'{graph_domain}{graph_version}{id}/insights?metric={metric}&access_token={access_token}'
    response = requests.get(link).json()
    id_videos.append(id)
    if (df9['Tipo'][i] == 'VIDEO') and (df9['Produto'][i] != 'REELS'):
        video_views.append(response['data'][0]['values'][0].get('value'))
    else:
        video_views.append(0)
            
df_videos = pd.DataFrame({'Id':id_videos,'Visualizações dos Vídeos':video_views})
df10 = pd.merge(df9,df_videos, on='Id', left_index=False)

print('Visualizações dos Vídeos coletados com sucesso !')

Visualizações dos Vídeos coletados com sucesso !


In [18]:
# Salvando a planilha principal após todas as coletas acima
df_main = df10.copy()

# Inserindo a coluna data da coleta dos dados no Dataframe principal
df_main['Data'] = day

df_main.to_csv(r'C:\Users\Ricardo\Desktop\tb_insta_main.csv', encoding='utf-8',index=False)
# df_main.to_csv(r'\\192.168.0.15\d$\Projects-Abendi\Marketing\Instagram\tb_insta_main.csv', encoding='utf-8',index=False)

## 7 - INSIGHTS DA CONTA (DIA)

**Requests API:**

https://graph.facebook.com/{api-version}/{ig-user-id}/insights?metric={metric}&period={period}&since={since}&until={until}&access_token={access-token}


In [19]:
metrics = 'follower_count,impressions,profile_views,reach'
link = f'{graph_domain}{graph_version}{ig_id}/insights?metric={metrics}&period=day&access_token={access_token}'

response = requests.get(link).json()
 
follower_count = response['data'][0]['values'][1].get('value')
impressions = response['data'][1]['values'][1].get('value')
profile_views = response['data'][2]['values'][1].get('value')
reach = response['data'][3]['values'][1].get('value')

'''
API Requests
https://graph.facebook.com/{api-version}/{ig-user-id}?fields={fields}&access_token={access-token}

'''
fields = 'followers_count,media_count'
link = f'{graph_domain}{graph_version}{ig_id}/?fields={fields}&access_token={access_token}'

response = requests.get(link).json()

followers_count = response['followers_count']
media_count = response['media_count']

df_geral_dia = pd.DataFrame({'Data': [day],'Novos Seguidores':[follower_count],'Impressões':[impressions],'Visualizações':[profile_views],
                            'Alcance':[reach],'Total Seguidores':followers_count,'Total Publicações':media_count}) 

print('Insights diários coletados com sucesso !')

Insights diários coletados com sucesso !


In [20]:
# Salvando dados coletados 
df_geral_dia.to_csv(r'C:\Users\Ricardo\Desktop\tb_insta_info_days.csv', encoding='utf-8', index=False)
# df_geral_dia.to_csv(r'\\192.168.0.15\d$\Projects-Abendi\Marketing\Instagram\tb_insta_info_days.csv', encoding='utf-8', index=False)

## 8 - DADOS DEMOGRÁFICOS

### 8.1 - IDADE E GENERO - TOTAL

In [21]:
metrics = 'audience_gender_age'
link = f'{graph_domain}{graph_version}{ig_id}/insights?metric={metrics}&period=lifetime&access_token={access_token}'

response = requests.get(link).json()

genero_idade = response['data'][0]['values'][0]['value']

indefinido = []
masculino = []
feminino = []

faixa_etaria = ['13-17','18-24','25-34','35-44','45-54','55-64','65+']

for key,value in genero_idade.items():
    if key.find('F.') and key.find('M.'):
        indefinido.append(value)
    elif key.find('F.') and key.find('U.'):
        masculino.append(value)
    else:
        feminino.append(value)
        
df_genero_idade = pd.DataFrame({'Data':day,'Faixa Etária':faixa_etaria,'Feminino':feminino,'Masculino':masculino,'Indefinido':indefinido})

df_genero_idade.to_csv(r'C:\Users\Ricardo\Desktop\tb_insta_age_gender.csv', encoding='utf-8',index=False)
# df_genero_idade.to_csv(r'\\192.168.0.15\d$\Projects-Abendi\Marketing\Instagram\tb_insta_age_gender.csv', encoding='utf-8',index=False)

print('Dados de Gênero e Idade coletados com sucesso')

Dados de Gênero e Idade coletados com sucesso


### 8.2 - CIDADES BRASILEIRAS / PAÍSES - TOTAL

In [7]:
metrics = 'audience_city,audience_country'
link = f'{graph_domain}{graph_version}{ig_id}/insights?metric={metrics}&period=lifetime&access_token={access_token}'

response = requests.get(link).json()

cidades_brasileiras = response['data'][0]['values'][0]['value']

cidade = []
estado = []
quantidade = []
cidade_estado = []

# Criando uma lista auxiliar 
for key,value in cidades_brasileiras.items():
    cidade_estado.append(key.split(','))

# Separando o estado da cidade
for i in range(len(cidade_estado)):
    cidade.append(cidade_estado[i][0])
    estado.append(cidade_estado[i][1])

# Removendo os caracteres (state)
for i in range(len(estado)):
    estado[i] = estado[i].replace('(state)','')
    
# Populando a lista de quantidade
for value in cidades_brasileiras.values():
    quantidade.append(value)
    
# Montando o Dataframe
df_cidades = pd.DataFrame({'Data':day,'Cidade':cidade,'Estado':estado,'Quantidade':quantidade})

df_cidades.to_csv(r'C:\Users\Ricardo\Desktop\tb_insta_cities.csv', encoding='utf-8', index=False)
# df_cidades.to_csv(r'\\192.168.0.15\d$\Projects-Abendi\Marketing\Instagram\tb_insta_cities.csv', encoding='utf-8', index=False)

print('Dados das cidades brasileiras coletados com sucesso !')

Dados das cidades brasileiras coletados com sucesso !


In [8]:
# Abrindo lista de códigos de países
codigos = pd.read_csv('codigo.txt', sep=',', encoding='latin-1', header=None)
codigos = codigos.rename(columns={0:'Código do País',1:'País'})

# Coletando dados dos países
paises = response['data'][1]['values'][0]['value']

# Separando código do país e a quantidade
codigo_pais = []
quantidade = []

for key,value in paises.items():
    codigo_pais.append(key)
    quantidade.append(value)

# Montando o dataframe
df_country = pd.DataFrame({'Data':day,'Código do País':codigo_pais,'Quantidade':quantidade})

# Juntando o df_country e o codigo através da coluna Id
df_country = pd.merge(df_country, codigos, on='Código do País', left_index=False)

df_country.to_csv(r'C:\Users\Ricardo\Desktop\tb_insta_countries.csv', encoding='utf-8', index=False)
# df_country.to_csv(r'\\192.168.0.15\d$\Projects-Abendi\Marketing\Instagram\tb_insta_countries.csv', encoding='utf-8', index=False)

print('Dados dos países coletados com sucesso !')

Dados dos países coletados com sucesso !


## 9 - BANCO DE DADOS

### 9.1 - Informações Gerais

Informações do Banco
- Banco de dados: PostgreSQL 14.3
- Usuário: postgres
- Senha: senha do usuário postgres

Tabelas Criadas:
- tb_insta_main
    - Data Coleta = data
    - Id = id
    - Link do Post = link_post
    - Descrição = descricao
    - Tipo = tipo
    - Data da Postagem = data_postagem
    - Likes = likes
    - Produto = produto
    - Engajamento = engajamento
    - Impressões = impressoes
    - Alcance = alcance
    - Salvos = salvos
    - Comentários = comentarios
    - Exibição dos Vídeos = exibicao_videos
    - Compartilhamentos = compartilhamentos
    - Total de Interações = total_interacoes
    - Engajamento do Carrossel = engajamento_carrossel
    - Impressões do Carrossel = impressoes_carrossel
    - Alcance do Carrossel = alcance_carrossel
    - Salvos do Carrossel = salvos_carrossel
    - Visualizações de Vídeo do Carrossel - videos_carrossel
    - Visualizações dos Vídeos = videos
- tb_insta_info_days
    - Data = data
    - Novos Seguidores = novos_seguidores
    - Impressões = impressoes
    - Visualizações = visualizoes
    - Alcance = alcance
    - Total Seguidores = total_seguidores
    - Total Publicações = total_publicacoes
- tb_insta_age_gender
    - Data = data
    - Faixa Etária = faixa
    - Feminino = feminino
    - Masculino = masculino
    - Indefinido = indefinido
- tb_insta_cities
    - Data = data
    - Cidade = cidade
    - Estado = estado
    - Quantidade = quantidade
- tb_insta_countries
    - Data = data
    - Código do País = codigo
    - Quantidade = quantidade
    - País = pais
- tb_stories
    - Data = data
    - Id = id do Story

### 9.2 - Populando as tabelas

In [ ]:
# Criando os dados para a conexão com o banco de dados PostgreSQL no servidor ABEDS01 (192.168.0.15)
dados_conexao = ('Driver=PostgreSQL Unicode(x64);Server=IP do servidor;Database=nome do banco;UID=postgres;PWD=senha do usuário')

# Criando a conexão com o banco de dados
conexao = pyodbc.connect(dados_conexao)

# Criando o cursor
cursor = conexao.cursor()

In [ ]:
# tb_insta_main
for i in df_main.index:
    data = df_main['Data'][i]
    id_ = df_main['Id'][i]
    link_post = df_main['Link do Post'][i]
    tipo = df_main['Tipo'][i]
    data_postagem = df_main['Data da Postagem'][i]
    likes = df_main['Likes'][i]
    produto = df_main['Produto'][i]
    engajamento = df_main['Engajamento'][i]
    impressoes = df_main['Impressões'][i]
    alcance = df_main['Alcance'][i]
    salvos = df_main['Salvos'][i]
    comentario = df_main['Comentários'][i]
    exibicao_videos = df_main['Exibição dos Vídeos'][i]
    compartilhamentos = df_main['Compartilhamentos'][i]
    total_interacoes = df_main['Total de Interações'][i]
    engajamento_carrossel = df_main['Engajamento do Carrossel'][i]
    impressoes_carrossel = df_main['Impressões do Carrossel'][i]
    alcance_carrossel = df_main['Alcance do Carrossel'][i]
    salvos_carrossel = df_main['Salvos do Carrossel'][i]
    videos_carrossel = df_main['Visualizações de Vídeo do Carrossel'][i]
    videos = df_main['Visualizações dos Vídeos'][i]

    # Inserção dos registros coletados na tabela tb_insta_main
    cursor.execute(f"INSERT INTO public.tb_insta_main(data, id_, link_post, tipo, data_postagem, likes, produto, engajamento, impressoes, alcance, salvos, comentario, exibicao_videos, compartilhamentos, total_interacoes, engajamento_carrossel, impressoes_carrossel, alcance_carrossel, salvos_carrossel, videos_carrossel, videos) VALUES ('{data}', '{id_}', '{link_post}', '{tipo}', '{data_postagem}', '{likes}', '{produto}', '{engajamento}', '{impressoes}', '{alcance}', '{salvos}', '{comentario}', '{exibicao_videos}', '{compartilhamentos}', '{total_interacoes}', '{engajamento_carrossel}', '{impressoes_carrossel}', '{alcance_carrossel}', '{salvos_carrossel}', '{videos_carrossel}', '{videos}');")

In [ ]:
# tb_insta_info_days

for i in df_geral_dia.index:
    data = df_geral_dia['Data'][i]
    novos_seguidores = df_geral_dia['Novos Seguidores'][i]
    impressoes = df_geral_dia['Impressões'][i]
    visualizacoes = df_geral_dia['Visualizações'][i]
    alcance = df_geral_dia['Alcance'][i]
    total_seguidores = df_geral_dia['Total Seguidores'][i]
    total_publicacoes = df_geral_dia['Total Publicações'][i]

    # Inserção dos registros coletados na tabela tb_insta_info_days
    cursor.execute(f"""
    INSERT INTO public.tb_insta_info_days(data,novos_seguidores,impressoes,visualizoes,alcance,total_seguidores,total_publicacoes)
    VALUES('{data}','{novos_seguidores}','{impressoes}','{visualizacoes}','{alcance}','{total_seguidores}','{total_publicacoes}');
    """)

In [ ]:
# tb_insta_age_gender
for i in df_genero_idade.index:
    data = df_genero_idade['Data'][i]
    faixa = df_genero_idade['Faixa Etária'][i]
    feminino = df_genero_idade['Feminino'][i]
    masculino = df_genero_idade['Masculino'][i]
    indefinido = df_genero_idade['Indefinido'][i]
    
    # Inserção dos registros coletados na tabela tb_insta_age_gender
    cursor.execute(f"""
    INSERT INTO public.tb_insta_age_gender(data,faixa,feminino,masculino,indefinido)
    VALUES('{data}','{faixa}','{feminino}','{masculino}','{indefinido}');
    """)

In [ ]:
# tb_insta_age_gender
for i in df_genero_idade.index:
    data = df_genero_idade['Data'][i]
    faixa = df_genero_idade['Faixa Etária'][i]
    feminino = df_genero_idade['Feminino'][i]
    masculino = df_genero_idade['Masculino'][i]
    indefinido = df_genero_idade['Indefinido'][i]
    
    # Inserção dos registros coletados na tabela tb_insta_age_gender
    cursor.execute(f"""
    INSERT INTO public.tb_insta_age_gender(data,faixa,feminino,masculino,indefinido)
    VALUES('{data}','{faixa}','{feminino}','{masculino}','{indefinido}');
    """)

In [ ]:
# tb_insta_countries
for i in df_country.index:
    data = df_country['Data'][i]
    codigo = df_country['Código do País'][i]
    quantidade = df_country['Quantidade'][i]
    pais = df_country['País'][i]

    # Inserção dos registros coletados na tabela tb_insta_countries
    cursor.execute(f"""
    INSERT INTO public.tb_insta_countries(data,codigo,quantidade,pais)
    VALUES('{data}','{codigo}','{quantidade}','{pais}');
    """)

In [ ]:
# tb_insta_stories
for i in df_stories.index:
    data = df_stories['Data'][i]
    id_ = df_stories['Id Stories'][i]

     # Inserção dos registros coletados na tabela tb_insta_countries 
    cursor.execute(f"""
    INSERT INTO public.tb_insta_stories(data,id_stories)
    VALUES('{data}','{id_}');
    """)   

In [ ]:
# Efetivando a inserção dos registros
cursor.commit()

In [ ]:
# Fechando a conexão com o Banco de Dados após a inserção dos registros
cursor.close()
conexao.close()

## 10 - NOTIFICANDO O RESPONSÁVEL

### INFORMAÇÕES GERAIS PARA ENVIO DE E-MAIL

    1 - Acesse a conta do Gmail que será utilizada para o envio do e-mail
    2 - Clique em "Gerenciar conta do Google" no menu do perfil
    3 - Clique em "Segurança"
    4 - Na opção "Como fazer login no Google" clique em "Senhas de App"
    5 - Clique em "Selecionar App" e escolha a opção "Outro - Nome Personalizado"
    6 - Defina um nome para o App e clique no botão GERAR
    7 - A senha que será utilizada para autenticação será exibida. Utilize esta senha na variável senha

In [ ]:
corpo_email = f"""
<p><b>***** Informativo *****</b></p>
<p>Rotina de carga de dados do Instagram realizada com sucesso no dia {day}.</p>
"""

msg = email.message.Message()
msg['Subject'] = "Carga de dados Instagram"
msg['From'] = remetente
msg['To'] = 'DIGITE O E-MAIL DO DESTINATÁRIO.'
password = senha 
msg.add_header('Content-Type', 'text/html')
msg.set_payload(corpo_email )

s = smtplib.SMTP('smtp.gmail.com: 587')
s.starttls()

s.login(msg['From'], password)
s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))

print('Email enviado')

## 11 - REQUIREMENTS.TXT

In [2]:
!pip3 freeze > requirements.txt